## Pregunta 2 – Extensión 1 (2,5 puntos).

Supón que un solo canal puede no ser suficiente para mover todos los datos entre ciertos
orígenes y destinos. Modifica las variables y restricciones para permitir más de un canal por par
origen–destino, manteniendo la capacidad de 10 MilGb por canal, y ajusta la función objetivo.

In [1]:
import gurobipy as gp
from gurobipy import GRB

In [3]:
# --- 1. DATOS DEL PROBLEMA ---

# Conjuntos de Orígenes (i) y Destinos (j)
Origenes = ['Lisboa', 'Madrid', 'Turin']
Destinos = ['Paris', 'Berlin', 'Varsovia']
Rutas = gp.tuplelist([(i, j) for i in Origenes for j in Destinos])

# Oferta de cada origen (en MilGb)
Oferta = {
    'Lisboa': 5,
    'Madrid': 6,
    'Turin': 7
}

# Demanda de cada destino (en MilGb)
Demanda = {
    'Paris': 4,
    'Berlin': 5,
    'Varsovia': 9
}

# Costos Unitarios de Transferencia (en €/MilGb)
Costo_Unitario_Centimos = {
    ('Lisboa', 'Paris'): 4, ('Lisboa', 'Berlin'): 3, ('Lisboa', 'Varsovia'): 6,
    ('Madrid', 'Paris'): 7, ('Madrid', 'Berlin'): 4, ('Madrid', 'Varsovia'): 9,
    ('Turin', 'Paris'): 9, ('Turin', 'Berlin'): 5, ('Turin', 'Varsovia'): 2
}
Costo_Unitario = {k: v / 100 for k, v in Costo_Unitario_Centimos.items()}

# Parámetros Operativos
COSTO_FIJO_BASE = 50.0  # Costo fijo por activar CADA canal base
CAPACIDAD_CANAL = 10.0  # Capacidad máxima de flujo por CADA canal
MAX_CANALES_BASE = 4  # Máximo de canales base disponibles

# Parámetros del Canal Adicional
COSTO_FIJO_ADICIONAL = 65.0
# La variable 'z' maneja el "a lo sumo un canal adicional"

In [4]:
# --- 2. CREACIÓN DEL MODELO ---
m = gp.Model("TransporteMultipleCanal")

Set parameter Username
Set parameter LicenseID to value 2718415
Academic license - for non-commercial use only - expires 2026-10-06


In [5]:
# --- 3. VARIABLES DE DECISIÓN ---

# x[i, j]: Flujo de datos (continuas, en MilGb)
x = m.addVars(Rutas, vtype=GRB.CONTINUOUS, name="Flujo")

# y[i, j]: Número de canales activados entre i y j (enteras, y_ij >= 0)
# Permite más de un canal por par origen-destino
y = m.addVars(Rutas, vtype=GRB.INTEGER, name="NumCanales")

# z: Alquiler del canal adicional (binaria)
z = m.addVar(vtype=GRB.BINARY, name="CanalAdicional")

In [6]:
# --- 4. FUNCIÓN OBJETIVO ---
# Minimizar Costo Total = Costo Variable + Costo Fijo (50€ * sum y_ij) + Costo Fijo Adicional (65€ * z)
Costo_Variable = gp.quicksum(Costo_Unitario[i, j] * x[i, j] for i, j in Rutas)
Costo_Fijo = gp.quicksum(COSTO_FIJO_BASE * y[i, j] for i, j in Rutas)
Costo_Fijo_Adicional = COSTO_FIJO_ADICIONAL * z

m.setObjective(Costo_Variable + Costo_Fijo + Costo_Fijo_Adicional, GRB.MINIMIZE)

In [7]:
# --- 5. RESTRICCIONES ---

# A. Restricciones de Oferta (Suministro)
m.addConstrs((x.sum(i, '*') == Oferta[i] for i in Origenes), name="Oferta")

# B. Restricciones de Demanda
m.addConstrs((x.sum('*', j) == Demanda[j] for j in Destinos), name="Demanda")

# C. Restricción de Enlace: Flujo total <= Capacidad total (x_ij <= 10 * y_ij)
# La capacidad es 10 MilGb * (Número de canales activados)
m.addConstrs((x[i, j] <= CAPACIDAD_CANAL * y[i, j]
              for i, j in Rutas), name="CapacidadTotalRuta")

# D. Restricción de Número Máximo de Canales
# El total de canales activados en la red (suma de todos los y_ij)
# está limitado por los 4 base más el adicional (z)
m.addConstr(y.sum() <= MAX_CANALES_BASE + z, name="MaximoCanalesRed")

# E. Restricción de Exclusión Mutua (Privacidad en Berlín)
# El número total de canales activados que llegan a Berlín desde Lisboa y Madrid
# no puede ser superior a 1 (siguiendo la interpretación estricta de la restricción original).
m.addConstr(y['Lisboa', 'Berlin'] + y['Madrid', 'Berlin'] <= 1, name="ExclusionMutua_Berlin")

<gurobi.Constr *Awaiting Model Update*>

In [8]:
# --- 6. OPTIMIZAR Y RESOLVER ---
m.optimize()

Gurobi Optimizer version 12.0.3 build v12.0.3rc0 (win64 - Windows 11.0 (26100.2))

CPU model: 12th Gen Intel(R) Core(TM) i7-1255U, instruction set [SSE2|AVX|AVX2]
Thread count: 10 physical cores, 12 logical processors, using up to 12 threads

Optimize a model with 17 rows, 19 columns and 48 nonzeros
Model fingerprint: 0x9353a533
Variable types: 9 continuous, 10 integer (1 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+01]
  Objective range  [2e-02, 7e+01]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 9e+00]
Presolve time: 0.00s
Presolved: 17 rows, 19 columns, 48 nonzeros
Variable types: 9 continuous, 10 integer (10 binary)
Found heuristic solution: objective 316.0500000
Found heuristic solution: objective 316.0100000

Root relaxation: objective 1.673367e+02, 12 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

   

In [13]:
# --- 7. IMPRIMIR RESULTADOS ---
if m.status == GRB.OPTIMAL:
    print("\n" + "=" * 60)
    print(" SOLUCIÓN ÓPTIMA ENCONTRADA (Múltiples Canales)")
    print(f"Costo Total Mínimo: {m.objVal:.2f} €")
    print("=" * 60)

    print("\n--- DETALLE DE FLUJOS Y CANALES ACTIVADOS ---")
    canales_totales = 0
    for i, j in Rutas:
        num_canales = int(y[i, j].x)
        if num_canales > 0:
            canales_totales += num_canales
            print(
                f"[{i} -> {j}]: Flujo = {x[i, j].x:.2f} MilGb | Canales = {num_canales} | Capacidad Máx = {num_canales * CAPACIDAD_CANAL:.0f} MilGb")

    print("\n--- RESUMEN DE COSTOS ---")

    costo_variable_final = sum(Costo_Unitario[i, j] * x[i, j].x for i, j in Rutas)
    costo_fijo_base_final = sum(COSTO_FIJO_BASE * y[i, j].x for i, j in Rutas)
    costo_fijo_adicional_final = z.x * COSTO_FIJO_ADICIONAL

    print(f"  Canales Base Activados (Σ y_ij): {canales_totales}")
    print(f"  Canal Adicional Alquilado (z): {'SÍ' if z.x > 0.5 else 'NO'}")
    print(f"  Costo Variable: {costo_variable_final:.2f} €")
    print(f"  Costo Fijo Base: {costo_fijo_base_final:.2f} € (50 € x {canales_totales})")
    print(f"  Costo Fijo Adicional: {costo_fijo_adicional_final:.2f} €")
    print(f"  Total: {costo_variable_final + costo_fijo_base_final + costo_fijo_adicional_final:.2f} €")

    print(
        f"\nNúmero total de canales utilizados: {canales_totales + int(z.x)} (Máximo permitido: {MAX_CANALES_BASE + int(z.x)})")


elif m.status == GRB.INFEASIBLE:
    print("El modelo no tiene solución factible.")


 SOLUCIÓN ÓPTIMA ENCONTRADA (Múltiples Canales)
Costo Total Mínimo: 200.75 €

--- DETALLE DE FLUJOS Y CANALES ACTIVADOS ---
[Lisboa -> Berlin]: Flujo = 5.00 MilGb | Canales = 1 | Capacidad Máx = 10 MilGb
[Madrid -> Paris]: Flujo = 4.00 MilGb | Canales = 1 | Capacidad Máx = 10 MilGb
[Madrid -> Varsovia]: Flujo = 2.00 MilGb | Canales = 1 | Capacidad Máx = 10 MilGb
[Turin -> Varsovia]: Flujo = 7.00 MilGb | Canales = 1 | Capacidad Máx = 10 MilGb

--- RESUMEN DE COSTOS ---
  Canales Base Activados (Σ y_ij): 4
  Canal Adicional Alquilado (z): NO
  Costo Variable: 0.75 €
  Costo Fijo Base: 200.00 € (50 € x 4)
  Costo Fijo Adicional: -0.00 €
  Total: 200.75 €

Número total de canales utilizados: 4 (Máximo permitido: 4)
